In [1]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

WORKING_DIR = '/Users/antho/Documents/WPI-MW'
FORECAST_SEASON = '25-26'
HINDCAST_SEASON = '24-25'


def assign_season_weight(season):
    if pd.isnull(season): return 1.0
    if season < "19-20": return 1.0
    if season == "19-20": return 1.0
    if season == "20-21": return 0.3
    if season == "21-22": return 0.7
    if season == "22-23": return 2.0
    if season == "23-24": return 3.0
    if season == "24-25": return 3.0
    return 1.0


def load_and_prepare_data():
    os.chdir(WORKING_DIR)
    history_df = pd.read_csv("anon_DataMerge.csv")
    event_manifest = pd.read_excel("EventManifest.xlsx", sheet_name="EventManifest")

    history_df.columns = history_df.columns.str.strip()
    history_df['EventDate'] = pd.to_datetime(history_df['EventDate'], errors='coerce')
    event_manifest['EventId'] = event_manifest['EventId']

    history_df = history_df.drop(columns=[
        'EventName', 'Season', 'EventType', 'EventStatus',
        'EventClass', 'EventVenue', 'EventGenre'
    ], errors='ignore')

    manifest_deduped = event_manifest.drop_duplicates(subset='EventId')

    merged_history = history_df.merge(
        manifest_deduped[['EventId', 'EventName', 'Season', 'EventType', 'EventStatus',
                          'EventClass', 'EventVenue', 'EventGenre']],
        on='EventId',
        how='left'
    )

    filtered_df = merged_history[
        (merged_history['EventType'] == 'Live') &
        (merged_history['EventStatus'] == 'Complete') &
        (merged_history['TicketStatus'] == 'Active') &
        (merged_history['TicketTotal'] > 0)
        ].copy()

    filtered_df['Weight'] = filtered_df['Season'].apply(assign_season_weight)
    filtered_df['WeightedTickets'] = filtered_df['Quantity'] * filtered_df['Weight']

    return event_manifest, merged_history, filtered_df


def compute_event_attendance(filtered_df):
    return (
        filtered_df
        .groupby(['EventId', 'EventName', 'EventClass', 'EventVenue', 'EventCapacity',
                  'EventGenre', 'Season'], group_keys=False)
        .agg(
            TotalTickets=('Quantity', 'sum'),
            WeightedTickets=('WeightedTickets', 'sum'),
            Weight=('Weight', 'first')
        )
        .reset_index()
    )


def compute_event_summary(event_attendance):
    return (
        event_attendance
        .groupby(['Season', 'EventName', 'EventVenue', 'EventCapacity'], group_keys=False)
        .agg(
            TotalAttendance=('TotalTickets', 'sum'),
            WeightedAttendance=('WeightedTickets', 'sum')
        )
        .reset_index()
        .sort_values('Season', ascending=False)
    )


def build_forecast_models(event_attendance):
    weighted_grouped = (
        event_attendance
        .groupby(['EventClass', 'EventVenue', 'EventGenre'], group_keys=False)
        .apply(lambda g: pd.Series({
            'WeightedAttendance': (g['TotalTickets'] * g['Weight']).sum(),
            'TotalWeight': g['Weight'].sum()
        }))
        .reset_index()
    )
    weighted_grouped['WeightedAvgAttendance'] = (
            weighted_grouped['WeightedAttendance'] / weighted_grouped['TotalWeight']
    )

    fallback_1 = (
        event_attendance
        .groupby(['EventClass', 'EventVenue'], group_keys=False)
        .apply(lambda g: pd.Series({
            'WeightedAttendance_f1': (g['TotalTickets'] * g['Weight']).sum(),
            'TotalWeight_f1': g['Weight'].sum()
        }))
        .reset_index()
    )
    fallback_1['WeightedAvgAttendance_f1'] = (
            fallback_1['WeightedAttendance_f1'] / fallback_1['TotalWeight_f1']
    )

    fallback_2 = (
        event_attendance
        .groupby(['EventGenre'], group_keys=False)
        .apply(lambda g: pd.Series({
            'WeightedAttendance_f2': (g['TotalTickets'] * g['Weight']).sum(),
            'TotalWeight_f2': g['Weight'].sum()
        }))
        .reset_index()
    )
    fallback_2['WeightedAvgAttendance_f2'] = (
            fallback_2['WeightedAttendance_f2'] / fallback_2['TotalWeight_f2']
    )

    return weighted_grouped, fallback_1, fallback_2


def forecast_attendance(event_manifest, weighted_grouped, fallback_1, fallback_2, season):
    forecast_df = event_manifest[
        (event_manifest['Season'] == season) &
        (event_manifest['EventType'].str.lower().str.strip() == 'live')
        ].drop_duplicates(subset='EventId').copy()

    for col in ['EventClass', 'EventVenue', 'EventGenre']:
        forecast_df[col] = forecast_df[col].astype(str).str.strip()

    forecast_df = forecast_df.merge(
        weighted_grouped[['EventClass', 'EventVenue', 'EventGenre', 'WeightedAvgAttendance']],
        on=['EventClass', 'EventVenue', 'EventGenre'],
        how='left'
    ).merge(
        fallback_1[['EventClass', 'EventVenue', 'WeightedAvgAttendance_f1']],
        on=['EventClass', 'EventVenue'],
        how='left'
    ).merge(
        fallback_2[['EventGenre', 'WeightedAvgAttendance_f2']],
        on='EventGenre',
        how='left'
    )

    forecast_df['WeightedAvgAttendance'] = forecast_df['WeightedAvgAttendance'].combine_first(
        forecast_df['WeightedAvgAttendance_f1']
    ).combine_first(
        forecast_df['WeightedAvgAttendance_f2']
    )

    forecast_df['PredictedAttendance'] = forecast_df.apply(
        lambda row: min(row['WeightedAvgAttendance'], row['EventCapacity'])
        if pd.notnull(row['WeightedAvgAttendance']) and pd.notnull(row['EventCapacity'])
        else row['WeightedAvgAttendance'],
        axis=1
    )

    return forecast_df[[
        'EventId', 'EventName', 'EventVenue', 'EventGenre', 'EventClass', 'EventDate',
        'EventCapacity', 'PredictedAttendance'
    ]]


def evaluate_hindcast(event_manifest, merged_history, weighted_grouped, fallback_1, fallback_2, season):
    hindcast_events = event_manifest[
        (event_manifest['Season'] == season) &
        (event_manifest['EventType'].str.lower().str.strip() == 'live')
        ].drop_duplicates(subset='EventId').copy()

    for col in ['EventClass', 'EventVenue', 'EventGenre']:
        hindcast_events[col] = hindcast_events[col].astype(str).str.strip()

    hindcast_forecast = hindcast_events.merge(
        weighted_grouped[['EventClass', 'EventVenue', 'EventGenre', 'WeightedAvgAttendance']],
        on=['EventClass', 'EventVenue', 'EventGenre'],
        how='left'
    ).merge(
        fallback_1[['EventClass', 'EventVenue', 'WeightedAvgAttendance_f1']],
        on=['EventClass', 'EventVenue'],
        how='left'
    ).merge(
        fallback_2[['EventGenre', 'WeightedAvgAttendance_f2']],
        on='EventGenre',
        how='left'
    )

    hindcast_forecast['WeightedAvgAttendance'] = hindcast_forecast['WeightedAvgAttendance'].combine_first(
        hindcast_forecast['WeightedAvgAttendance_f1']
    ).combine_first(
        hindcast_forecast['WeightedAvgAttendance_f2']
    )

    hindcast_forecast['PredictedAttendance'] = hindcast_forecast.apply(
        lambda row: min(row['WeightedAvgAttendance'], row['EventCapacity'])
        if pd.notnull(row['WeightedAvgAttendance']) and pd.notnull(row['EventCapacity'])
        else row['WeightedAvgAttendance'],
        axis=1
    )

    hindcast_summary = (
        hindcast_forecast
        .groupby('EventName', group_keys=False)
        .agg(PredictedAttendance=('PredictedAttendance', 'sum'))
        .reset_index()
    )

    actuals = (
        merged_history[
            (merged_history['Season'] == season) &
            (merged_history['EventType'] == 'Live') &
            (merged_history['EventStatus'] == 'Complete') &
            (merged_history['TicketStatus'] == 'Active') &
            (merged_history['TicketTotal'] > 0)
            ]
        .groupby('EventName', group_keys=False)
        .agg(ActualAttendance=('Quantity', 'sum'))
        .reset_index()
    )

    comparison = hindcast_summary.merge(actuals, on='EventName', how='outer')
    comparison['AbsoluteError'] = (comparison['ActualAttendance'] - comparison['PredictedAttendance']).abs()
    comparison['PercentError'] = (
            comparison['AbsoluteError'] / comparison['ActualAttendance']
    ).where(comparison['ActualAttendance'] > 0)

    valid_rows = comparison[comparison['ActualAttendance'] > 0]
    mape = valid_rows['PercentError'].mean() * 100
    wape = (valid_rows['AbsoluteError'].sum() / valid_rows['ActualAttendance'].sum()) * 100

    summary_df = pd.DataFrame([
        {'Metric': 'MAPE', 'Value': f"{mape:.2f}%"},
        {'Metric': 'WAPE', 'Value': f"{wape:.2f}%"}
    ])

    output_file = f"Forecast_Evaluation_{season.replace('-', '')}.xlsx"
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        comparison.to_excel(writer, sheet_name="Forecast_vs_Actuals", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)

    print(f"✅ Hindcast evaluation for {season} written to {output_file}")


def main():
    event_manifest, merged_history, filtered_df = load_and_prepare_data()
    event_attendance = compute_event_attendance(filtered_df)
    event_summary = compute_event_summary(event_attendance)
    weighted_grouped, fallback_1, fallback_2 = build_forecast_models(event_attendance)

    forecast = forecast_attendance(event_manifest, weighted_grouped, fallback_1, fallback_2, FORECAST_SEASON)

    with pd.ExcelWriter("Forecast_25_26_Simplified.xlsx", engine="openpyxl") as writer:
        forecast.to_excel(writer, sheet_name="Forecast_25_26", index=False)
        event_summary.to_excel(writer, sheet_name="HistoricalAttendance", index=False)

    print("✅ Forecast and attendance summary written to Forecast_25_26_Simplified.xlsx")

    evaluate_hindcast(event_manifest, merged_history, weighted_grouped, fallback_1, fallback_2, HINDCAST_SEASON)


if __name__ == "__main__":
    main()

✅ Forecast and attendance summary written to Forecast_25_26_Simplified.xlsx
✅ Hindcast evaluation for 24-25 written to Forecast_Evaluation_2425.xlsx
